This file instantiates objects of the class Particle in the particle.ipynb file, using initial velocities and initial positions generated in the sampling.ipynb file.

Author: 18BME2104 Kaushal

In [4]:
import numpy as np # For computations
import csv # To read csv file of position and velocities required to initialize particles
import import_ipynb
import particle
import field

importing Jupyter notebook from particle.ipynb
Particle describes a particle in the plasma         
 A particle has:         
 mass: m         
 charge: q         
 position: r = [x, y, z]         
 velocity: v = [v_x, v_y, v_z]         
 acceleration: a = [a_x, a_y, a_z]         
importing Jupyter notebook from field.ipynb


In [ ]:
# Probably good idea to make plots, write to csv and other things in another notebook. Let's see.

In [14]:
class Step:
    '''
    An instance object of the step class can:
    1. initialize particles with positions and velocities passed from within the setup or
    to be read from saved files
    2. update each particle under the action of magnetic and electric fields
    
    SUGGESTIONS FOR IMPROVEMENT
    Could be extended to:
    1. Save particle positions and velocities after a period of simulation
    '''
    
    def __init__(self, file_with_filenames = \
                 '/home/kushik/Kushik/VIT/Eighth semester/MagneticMirror/csvfiles/available files.csv'):
        self.file_with_filenames = file_with_filenames
        '''available files.csv contains the files from which positions and velocities
        could be read
        Optionally they could just be generated during the program using sampling methods
        in the Sampling class in sampling.ipynb
        However for better performance, it might be better to sample positions and velocities
        and store them in csv files
        and later read them from the files and use them.
        Sampled positions and velocities can also be reused in this way.
        ''' 
        
        self.filenames, self.r_filenames, self.v_filenames = self.read_available_filenames(self.file_with_filenames)
        '''self.filenames would then be a list of availble files to read from
           self.r_filenames would be the list of available files with position written on them
           self.v_filenames would be the list of available files with velocities written on them'''
        
        self.particles = []
        # The list of the particles created during an instance of a Step object.
        # Or one static state of number of particles, in a dynamic running of plasma
        
        self.number_of_files_available = len(self.filenames)
        self.filenames_descriptions = dict(zip(self.filenames,['' for i in range(self.number_of_files_available)]))   
        #Holds descriptions of files stored in the list filenames 
        #like Temperature for the Maxwellian sampling methods
        
    def __string__(self):
        '''
        As string for an instance of the Step class has not been defined.
        However a paramter like the starting time of this step, number of particles in this step, etc.
        could be printed for information
        '''
        pass
            
    ### FILE READING SECTION
        
    def read_available_filenames(self):
        '''
        Reads the names of the available files from available files.csv file 
        from the directory (including the file path) self.file_with_filenames 
        
        Arguments:
        self
        
        Returns:
        filenames: Names of the available files
        r_filenames: Names of the available files with positions of particles
        v_filenames: Names of the available files with velocities of particles
        '''
        
        
        rows = []
        filenames = []
        r_filenames = []
        v_filenames = []
        with open(self.file_with_filenames, 'r') as csvfile:
            # creating a csv reader object
            csvreader = csv.reader(csvfile)

            # extracting field names through first row
            #fields = next(csvreader)

            # extracting each data row one by one
            for row in csvreader:
                rows.append(row)
        for row in rows:
            empty = ''
            for char in row:
                empty += char
            filenames.append(empty)
        for filename in filenames:
            r_or_v = filename.split()[1]
            if r_or_v == 'r':
                r_filenames.append(filename)
            else:
                if r_or_v == 'v':
                    v_filenames.append(filename)
                else:
                    raise Exception('There might be some error in filename or how filename is processed in \
                    file reading functions')
    
        return filenames, r_filenames, v_filenames
    
    def print_available_files(self):
        '''
        Prints the names of the available files from self.filenames
        
        Arguments:
        self
        
        Returns:
        nothing
        '''
        
        
        print(self.filenames)
    
    def print_available_r_files(self):
        '''
        Prints the names of the available files with positions from self.r_filenames
        
        Arguments:
        self
        
        Returns:
        nothing
        '''
        
        
        print(self.r_filenames)
        
    def print_available_v_filenames(self):
        '''
        Prints the names of the available files with velocities from self.v_filenames
        
        Arguments:
        self
        
        Returns:
        nothing
        '''
        
        
        print(self.v_filenames)       
    
    def read_r_or_v_file(self, index):
        '''
        Reads a file from self.filnames of available files to read positions or velocities from
        print_available_files can be called to check the available files and determine the index to be passed in
        
        Arguments:
        self
        index: the index on the self.filenames list of which the corresponding file is to be read.
        
        Returns:
        a list of the rows of the file read
        '''
        
        
        rows = []
        with open(self.filenames[index], 'r') as csvfile:
            csvreader = csv.reader(csvfile)

            for row in csvreader:
                rows.append(row)
        return rows
    
    def read_r_file(self, index):
        '''
        Reads a file from self.r_filnames of available files to read positions from
        print_available_r_files can be called to check the available files and determine the index to be passed in
        
        Arguments:
        self
        index: the index on the self.r_filenames list of which the corresponding file is to be read.
        
        Returns:
        a list of the rows of the file read
        '''
        
        
        rows = []
        with open(self.r_filenames[index], 'r') as csvfile:
            csvreader = csv.reader(csvfile)

            for row in csvreader:
                rows.append(row)
        return rows
    
    def read_v_file(self, index):
        '''
        Reads a file from self.r_filnames of available files to read velocities from
        print_available_v_files can be called to check the available files and determine the index to be passed in
        
        Arguments:
        self
        index: the index on the self.v_filenames list of which the corresponding file is to be read.
        
        Returns:
        a list of the rows of the file read
        '''
        
        
        rows = []
        with open(self.filenames[index], 'r') as csvfile:
            csvreader = csv.reader(csvfile)

            for row in csvreader:
                rows.append(row)
        return rows
    
    def reshaper(self, array_from_file):
        '''
        Reshapes a list for example, the list returned by read_r_or_v_file, read_r_file or read_v_file 
        into lists of positions or velocities of particles (3 components) 
        
        Arguments:
        self
        array_from_file: array to be reshaped, 
        for example the list returned by read_r_or_v_file, read_r_file or read_v_file
        
        Returns:
        An array reshaped into a list of position or velocity - like arrays (with 3 components) 
        '''
        
        
        three_n = len(array_from_file)
        n = int(three_n / 3)
        reshaped_array = np.array(array_from_file).reshape((n, 3))
        return reshaped_array
        
    def read_r_or_v_file_and_reshape(self, index):
        '''
        Reads file on the index: index, of seld.filenames 
        and returns a list of position or velocity -like shaped arrays 
        
        Arguments:
        self
        index: the index on the self.filenames list of which the corresponding file is to be read.
        
        Returns:
        An array reshaped into a list of position or velocity - like arrays (with 3 components)
        '''
        
        
        array_from_file = self.read_r_or_v_file(index)
        reshaped_array = self.reshaper(array_from_file)
        return reshaped_array
        
    def read_r_file_and_reshape(self, index):
        '''
        Reads file on the index: index, of seld.r_filenames 
        and returns a list of position -like shaped arrays 
        
        Arguments:
        self
        index: the index on the self.r_filenames list of which the corresponding file is to be read.
        
        Returns:
        An array reshaped into a list of position - like arrays (with 3 components)
        '''
        
        
        array_from_file = self.read_r_file(index)
        reshaped_array = self.reshaper(array_from_file)
        return reshaped_array
    
    def read_v_file_and_reshape(self, ,index):
        '''
        Reads file on the index: index, of seld.v_filenames 
        and returns a list of position -like shaped arrays 
        
        Arguments:
        self
        index: the index on the self.v_filenames list of which the corresponding file is to be read.
        
        Returns:
        An array reshaped into a list of velocity - like arrays (with 3 components)
        '''
        
        
        array_from_file = self.read_v_file(index)
        reshaped_array = self.reshaper(array_from_file)
        return reshaped_array
    
    def add_description_to_available_files(self, index, description):
        '''
        Adds description in self.filenames_description corresponding to 
        a file at the index: index in self.filenames 
        
        Arguments:
        self
        index: the index corresponding to the file in self.filenames whose description is to be changed/added
        description: the description to be added, a string.
        
        Returns:
        nothing
        '''
        
        
        self.filenames_descriptions[ self.filenames[index] ] = description
    
    ### PARTICELS INTIALIZATION SECTION
    def initialize_particles(self, names, q_s, m_s, r_0_s, v_0_s, a_0_s, n):
        '''
        Instantiates objects of the Particle class from particle.ipynb and adds them to the list self.particles
        
        Arguments:
        self
        names, q_s, m_s, r_0_s, v_0_s, a_0_s are arrays of name, q, m, r_0, v_0, a_0
        respectively for each particle arranged such that for all of these arrays
        the i_th index represents q, m, r_0, v_0, a_0 of the i_th particle
        n: number of particles initialized, or the length of each of these arrays which is the same
        
        Returns:
        nothing
        '''
        
        
        for i in range(n):
            self.particles.append(Particle(names[i], q_s[i], m_s[i], r_0_s[i], v_0_s[i], a_0_s[i] ))
        ### Might need to adjust for a_0_s[i]
    
    ### FIELDS INITIALIZATION SECTION
    def initialize_fields(self):
        '''
        Instantiates an object of the Field class from field.ipynb and assigns it to self.fields
        
        Arguments:
        self
        
        Returns:
        nothing
        '''
        
        
        self.fields = Field()
    
    ### TIME STEP SECTION
    def update_particles(self, dt, argsE, argsB):
        '''
        Updates each particle in self.particles
        
        Arguments:
        self
        dt: time step of update
        argsE: arguments required for generating the electric field at each particle's position
        required to move the particles, by calling the method get_E_field which is passed
        to the update method of each particle object in self.particles
        argsB: arguments required for generating the magnetic field at each particle's position
        required to move the particles, by calling the method get_B_field which is passed
        to the update method of each particle object in self.particles
        
        Returns:
        nothing
        '''
        
        
        for particle in self.particles:
            particle.update(self.fields, dt, argsE, argsB)
    

In [ ]:
#create many particles
# by sampling different positions and velocities using various distribution functions
#Loop
    #get fields at their position
    #move the particles
    
                ''''THIS IN run.ipynb
                '''#create new set of particles after some time, to model flow of particles/ionized gas 
                #in a magnetron sputtering chamber

                #For old particles that move out of the trap bounds (position exceeding some limits), or 
                #modelingf particles being absorbed in the sputtered coating
                #with an associated probability of absorption
                #just delete those particles from the collection
                #(but can be saved in csv files, so that they can stay recorded to be reused later)